In [ ]:
#Необходимые импорты
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, max_error
from sklearn import metrics

In [1]:
#Визуальная оценка датасета
df = pd.read_csv('dataset2.csv') 
df #k = columns/3

NameError: name 'pd' is not defined

In [ ]:
# Подбор признаков для классификации и регрессии
Y_cls = df['IHD']
X_cls = df.drop(columns = ['IHD', 'SL'])
Y_reg = df['SL']
X_reg = df.drop(columns = ['IHD', 'SL'])

# 20 признаков для классификации
selector_cls = SelectKBest(chi2, k = n)
X_new = selector_cls.fit_transform(X_cls*1, Y_cls)
features_cls = X_cls.columns[selector_cls.get_support()]
print(features_cls)

# 20 признаков для регрессии
selector_reg = SelectKBest(f_regression, k = n)
X_new = selector_reg.fit_transform(X_reg*1, Y_reg)
features_reg = X_reg.columns[selector_reg.get_support()]
print(features_reg)

In [2]:
# Вывод графика выбранных признаков для классификации
# Красным показаны значения p-values для 13 признаков, которые мы выбрали
# Синим выделены признаки, которые мы не выбираем

mask = selector_cls.get_support()
scores = -np.log10(selector_cls.pvalues_)
X_indices = np.arange(X_cls.shape[-1])
plt.figure(1, figsize=[15, 7])
plt.clf()
plt.bar(X_indices[mask], scores[mask] ** 0.5, width=0.2, color = 'red')
plt.bar(X_indices[~mask], scores[~mask] ** 0.5, width=0.2, color = 'blue')
plt.show()
X_cls = X_cls.loc[:, mask]

NameError: name 'selector_cls' is not defined

In [ ]:
# Вывод графика выбранных признаков для регрессии
mask = selector_reg.get_support()
scores = -np.log10(selector_reg.pvalues_)
X_indices = np.arange(X_reg.shape[-1])
plt.figure(1, figsize=[15, 7])
plt.clf()
plt.bar(X_indices[mask], scores[mask] ** 0.5, width=0.2, color = 'red')
plt.bar(X_indices[~mask], scores[~mask] ** 0.5, width=0.2, color = 'blue')
plt.show()
X_reg = X_reg.loc[:, mask]

In [ ]:
#PCA-анализ или уменьшение размерности для классификации
X = X_cls.to_numpy()
X = X.astype(float)
pca = decomposition.PCA(n_components=2)
X_centered = X
X_centered = (X - X.mean(axis=0)) / np.std(X, axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)
# p - признак, по которому выводим цвет, синий - ближе к минимальному значению, красный - к максимальному
p = 0
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(10, 10)
axis.scatter(X_pca[:, 0], X_pca[:, 1], s=10, c=(X[:, p] - np.min(X[:, p])) /  (np.max(X[:, p])- np.min(X[:, p])), cmap='jet')
plt.show()

for i, component in enumerate(pca.components_):
    print("{} component: {}% of initial variance".format(i + 1, round(100 * pca.explained_variance_ratio_[i], 2)))
    print(" + ".join("%.3f x %s" % (value, name) for value, name in zip(component, X_cls.columns)))

In [ ]:
#Вывод графика после уменьшения размерности
p = 0
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(10, 10)
axis.scatter(X_pca[:, 0], X_pca[:, 1], s = 10 + 50*Y_cls, c =(X[:, p] - np.min(X[:, p])) /  (np.max(X[:, p])- np.min(X[:, p])), cmap='jet')
plt.show()

In [ ]:
#Вывод матрицы корреляций после уменьшения размерности
X_cls_corr = X_cls.corr()
fig, ax = plt.subplots(figsize=(15, 10)) 
sns.heatmap(X_cls_corr, ax=ax, xticklabels=True, yticklabels=True)

In [ ]:
#PCA-анализ или уменьшение размерности для регрессии
X = X_reg.to_numpy()
X = X.astype(float)
pca = decomposition.PCA(n_components=2)
X_centered = X
X_centered = (X - X.mean(axis=0)) / np.std(X, axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)
# p - признак, по которому выводим цвет, синий - ближе к минимальному значению, красный - к максимальному
p = 0
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(10, 10)
axis.scatter(X_pca[:, 0], X_pca[:, 1], s=10, c=(X[:, p] - np.min(X[:, p])) /  (np.max(X[:, p])- np.min(X[:, p])), cmap='jet')
plt.show()

for i, component in enumerate(pca.components_):
    print("{} component: {}% of initial variance".format(i + 1, round(100 * pca.explained_variance_ratio_[i], 2)))
    print(" + ".join("%.3f x %s" % (value, name) for value, name in zip(component, X_reg.columns)))

In [ ]:
#Вывод графика после уменьшения размерности
p = 0
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(10, 10)
temp = Y_reg.copy()
temp[temp == -1] = float("+inf")
temp[temp < 2] = 2
temp = 1 / np.log2(temp)
indx = temp > 0.3
axis.scatter(X_pca[indx, 0], X_pca[indx, 1], s = X[indx, p], c = temp[indx], cmap='copper')
plt.show()

In [ ]:
#Вывод матрицы корреляций после уменьшения размерности
X_reg_corr = X_reg.corr()
fig, ax = plt.subplots(figsize=(15, 10)) 
sns.heatmap(X_reg_corr, ax=ax, xticklabels=True, yticklabels=True)

In [3]:
#Определение дисбаланса данных
print([sum(Y_cls), len(Y_cls)]) 
# https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data
print([sum(Y_reg==-1), sum(Y_reg==0), sum(Y_reg==1), sum((Y_reg>=0)*(Y_reg<=360)), sum(Y_reg>360), max(Y_reg), Y_reg.quantile(0.95), len(Y_reg)])
plt.plot(Y_reg.sort_values().values)

In [ ]:
temp = Y_reg.copy()
temp[temp == -1] = float("+inf")
temp[temp < 2] = 2
temp = 1 / np.log2(temp)
Y_reg = temp
plt.plot(Y_reg.sort_values().values)

In [ ]:
indx_1 = Y_cls == 1
indx_0 = Y_cls == 0
# Удаляем строчки с мажоритарным классом с вероятностью (1 - sum(Y_cls) / len(Y_cls))
counter = 0
for i in range(len(Y_cls)):
    if indx_0[i] == True:
        counter = counter + 1
    if counter > 2741:
        indx_0[i] = False

X_train_cls1, X_test_cls1, y_train_cls1, y_test_cls1 = train_test_split(X_cls[indx_1], Y_cls[indx_1], test_size=0.1, random_state=1)
X_train_cls1, X_val_cls1, y_train_cls1, y_val_cls1 = train_test_split(X_train_cls1, y_train_cls1, test_size=0.2, random_state=1)

X_train_cls0, X_test_cls0, y_train_cls0, y_test_cls0 = train_test_split(X_cls[indx_0], Y_cls[indx_0], test_size=0.24, random_state=1)
X_train_cls0, X_val_cls0, y_train_cls0, y_val_cls0 = train_test_split(X_train_cls0, y_train_cls0, test_size=0.63, random_state=1)

X_train_cls = pd.concat([X_train_cls0, X_train_cls1])
y_train_cls = pd.concat([y_train_cls0, y_train_cls1])
X_val_cls = pd.concat([X_val_cls0, X_val_cls1])
y_val_cls = pd.concat([y_val_cls0, y_val_cls1])
X_test_cls = pd.concat([X_test_cls0, X_test_cls1])
y_test_cls = pd.concat([y_test_cls0, y_test_cls1])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, Y_reg, test_size=0.1, random_state=1) # 0.2 тестовая выборка
X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(X_train_reg, y_train_reg, test_size=0.2, random_state=1) # 0.2*0.8 = 0.18 валидационная, 0.72 обучающая

In [ ]:
# Выбор модели прогнозирования и настройка гиперпараметров для регрессии
names = [
    "LinearRegression",
    "Ridge",
    "BayesianRidge",
    "PolynomialFeatures",
    "SVR linear",
    "SVR rbf",
    "KNeighborsRegressor",
    "GaussianProcessRegressor",
    "DecisionTreeRegressor",
    "RandomForestRegressor",
    "MLPRegressor",
]

regressors = [
    LinearRegression(),
    Ridge(alpha=1.0, solver='auto'),
    BayesianRidge(),
    Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))]),
    SVR(kernel='linear'),
    SVR(kernel='rbf'),
    KNeighborsRegressor(3),
    GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel()),
    DecisionTreeRegressor(max_depth=5),
    RandomForestRegressor(max_depth=5, n_estimators=10, max_features=2),
    MLPRegressor(alpha=1, max_iter=1000),
]

for name, rgr in zip(names, regressors):
    rgr.fit(X_train_reg, y_train_reg)
    pred_reg = rgr.predict(X_val_reg)
    score = mean_squared_error(y_val_reg, pred_reg)
    print(name, score)
    
# Пороговое значение MSE, когда в качестве y выбираем среднее значение по y
np.mean((np.mean(y_val_reg)-y_val_reg) **2)

In [ ]:
#Выбираем следующие модели для подбора гиперпараметров: LinearRegression (без настройки) 
#BayesianRidge KNeighborsRegressor DecisionTreeRegressor RandomForestRegressor MLPRegressor
parameters = {'max_depth':[13], 'n_estimators':[200], 'max_features': ['sqrt']}

rgr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=parameters, scoring='neg_mean_squared_error')
rgr.fit(X_train_reg, y_train_reg)

results = pd.DataFrame.from_dict(rgr.cv_results_)
results = results.sort_values('rank_test_score', ignore_index=True)
print(results['params'][0])
pred_reg = rgr.predict(X_val_reg)

print(mean_squared_error(y_val_reg, pred_reg))
print(r2_score(y_val_reg, pred_reg))
print(mean_absolute_error(y_val_reg, pred_reg))
print(max_error(y_val_reg, pred_reg))

In [ ]:
parameters = {'n_neighbors':[15, 25, 50], 'weights':['uniform', 'distance'], 'p':[1, 2]}

rgr = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=parameters, scoring='neg_mean_squared_error')
rgr.fit(X_train_reg, y_train_reg)

results = pd.DataFrame.from_dict(rgr.cv_results_)
results = results.sort_values('rank_test_score', ignore_index=True)
print(results['params'][0])
pred_reg = rgr.predict(X_val_reg)

print(mean_squared_error(y_val_reg, pred_reg))
print(r2_score(y_val_reg, pred_reg))
print(mean_absolute_error(y_val_reg, pred_reg))
print(max_error(y_val_reg, pred_reg))

In [ ]:
parameters = {'n_iter':[500], 'tol':[1e-4], 'alpha_1':[1e-5, 1e-6, 1e-7], 'alpha_2':[1e-5, 1e-6, 1e-7]}

rgr = GridSearchCV(estimator=BayesianRidge(), param_grid=parameters, scoring='neg_mean_squared_error')
rgr.fit(X_train_reg, y_train_reg)

results = pd.DataFrame.from_dict(rgr.cv_results_)
results = results.sort_values('rank_test_score', ignore_index=True)
print(results['params'][0])
pred_reg = rgr.predict(X_val_reg)

print(mean_squared_error(y_val_reg, pred_reg))
print(r2_score(y_val_reg, pred_reg))
print(mean_absolute_error(y_val_reg, pred_reg))
print(max_error(y_val_reg, pred_reg))

In [ ]:
parameters = {'hidden_layer_sizes':[(20, ), (50, ), (100, )], 'activation':['relu', 'logistic'], 'solver':['adam', 'lbfgs']}

rgr = GridSearchCV(estimator=MLPRegressor(), param_grid=parameters, scoring='neg_mean_squared_error')
rgr.fit(X_train_reg, y_train_reg)

results = pd.DataFrame.from_dict(rgr.cv_results_)
results = results.sort_values('rank_test_score', ignore_index=True)
print(results['params'][0])
pred_reg = rgr.predict(X_val_reg)

print(mean_squared_error(y_val_reg, pred_reg))
print(r2_score(y_val_reg, pred_reg))
print(mean_absolute_error(y_val_reg, pred_reg))
print(max_error(y_val_reg, pred_reg))